# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,luderitz,-26.6481,15.1594,286.81,90,1,5.09,NaN,1723415588
1,1,port-aux-francais,-49.3500,70.2167,277.98,96,100,15.31,TF,1723415589
2,2,kailua-kona,19.6406,-155.9956,303.86,76,0,6.69,US,1723415591
3,3,prado,-17.3411,-39.2208,294.84,87,100,2.91,BR,1723415592
4,4,olonkinbyen,70.9221,-8.7187,275.88,87,100,4.82,SJ,1723415593


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [38]:

# # Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = 0.85,
    color = 'City'
)

# # Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [61]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Humidity'] <= 30)  & (city_data_df['Max Temp'] <= 300)] 

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,puerto quijarro,-17.7833,-57.7667,294.51,27,67,2.40,BO,1723415598
48,48,otjimbingwe,-22.3500,16.1333,291.24,12,40,0.19,NaN,1723415645
85,85,suncho corral,-27.9336,-63.4294,288.42,21,0,4.27,AR,1723415690
136,136,ghanzi,-21.5667,21.7833,289.69,12,1,3.41,BW,1723415755
215,215,siyabuswa,-25.1132,29.0445,289.70,12,57,2.94,ZA,1723415848
223,223,pedregulho,-20.2569,-47.4767,286.93,26,6,2.49,BR,1723415858
278,278,san jose de jachal,-30.2406,-68.7469,283.64,22,0,1.79,AR,1723415924
384,384,tupiza,-21.4500,-65.7167,281.14,11,100,3.57,BO,1723416047
428,428,hacienda huancane,-15.8222,-70.8856,278.12,21,82,5.36,PE,1723416098
481,481,rakops,-21.0167,24.3333,291.27,15,0,0.76,BW,1723416160


### Step 3: Create a new DataFrame called `hotel_df`.

In [69]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.loc[:,['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
9,puerto quijarro,BO,-17.7833,-57.7667,27,
48,otjimbingwe,NaN,-22.3500,16.1333,12,
85,suncho corral,AR,-27.9336,-63.4294,21,
136,ghanzi,BW,-21.5667,21.7833,12,
215,siyabuswa,ZA,-25.1132,29.0445,12,
223,pedregulho,BR,-20.2569,-47.4767,26,
278,san jose de jachal,AR,-30.2406,-68.7469,22,
384,tupiza,BO,-21.4500,-65.7167,11,
428,hacienda huancane,PE,-15.8222,-70.8856,21,
481,rakops,BW,-21.0167,24.3333,15,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = # YOUR CODE HERE

    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
badger - nearest hotel: Clarion Hotel & Suites
ust'-kulom - nearest hotel: No hotel found
emerald - nearest hotel: Emerald Central Hotel
newman - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
laguna - nearest hotel: Holiday Inn Express & Suites
nova vicosa - nearest hotel: Pousada Pontal da Barra
fort myers shores - nearest hotel: Comfort Inn Fort Myers
trinity beach - nearest hotel: Blue Lagoon Resort
racine - nearest hotel: No hotel found
six-fours-les-plages - nearest hotel: Kyriad Direct La Seyne-sur-Mer
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho
posto fiscal rolim de moura - nearest hotel: No hotel found
port douglas - nearest hotel: Mantra Aqueous Port Douglas
roswell - nearest hotel: Best Western Plus Hotel
hell-ville - nearest hotel: ABUD HOTEL
sona - nearest hotel: Al Bosco
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by W

,City,Country,Lat,Lng,Humidity,Hotel Name
43,saint-pierre,RE,-21.3393,55.4781,61,Tropic Hotel
66,badger,US,64.8000,-147.5333,37,Clarion Hotel & Suites
102,ust'-kulom,RU,61.6864,53.6902,43,No hotel found
120,emerald,AU,-23.5333,148.1667,28,Emerald Central Hotel
130,newman,US,37.3138,-121.0208,51,No hotel found
138,fort bragg,US,35.1390,-79.0060,80,Airborne Inn Lodging
146,laguna,US,38.4210,-121.4238,52,Holiday Inn Express & Suites
160,nova vicosa,BR,-17.8919,-39.3719,87,Pousada Pontal da Barra
170,fort myers shores,US,26.7092,-81.7459,95,Comfort Inn Fort Myers
176,trinity beach,AU,-16.7888,145.6968,44,Blue Lagoon Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)